![DSB logo](img/Dolan.jpg)
# Apply Functions to Your DataFrame

## PD4E Chapter 9: Apply
### How do you read/manipulate/store data in Python?

# What You Learned in Python/Pandas that could Apply Here

You will need following knowledge from the first half of this course:
1. functions
2. subsetting/slicing data
3. Loops

# What You will Learn in this Chapter

You will learn following techniques in this chapter:
1. how to apply functions to columns, rows, or the whole DataFrame
2. Different use cases between `.apply()`, `.map()`, and `.applymap()`
3. `lambda` - the nameless, defintion-less functions

# Review of Functions

- Functions are __reusable__ code blocks 
    - where we group some statements together
- In `pandas`, we use functions a lot, particularly in the data preprocessing step
    - e.g., write a function to calculate some values, for consistency we want to use it to all applicable columns
- Functions can be categorized as _fruitful_ and _void_
    - here we mostly care about _fruitful_ functions

In [1]:
# example of a fruitful function
def avg_2(x, y = 10):
    return (x + y) / 2

avg_2(4)

7.0

# Why `.apply()`?

- when you want to use a function on a DataFrame, directly calling the function on it, or its columns will  actually work
    - but sometimes it does not work as we expected
- consider `.apply()` as `pandas` way of calling functions
    - note that you still have to define your function 

In [2]:
# an example
import pandas as pd

df1=pd.DataFrame({'a':[10,20,30],
                 'b':[20,30,40]})
df1

,a,b
0,10,20
1,20,30
2,30,40


In [3]:
# function def. - calculate square
def my_sq(x):
    return x ** 2

In [4]:
# let's try calling the function the normal way
my_sq(df1['a'])

0    100
1    400
2    900
Name: a, dtype: int64

In [5]:
# how about the whole DF?
my_sq(df1)

,a,b
0,100,400
1,400,900
2,900,1600


# What happened above?

- Looks like we can call the function (`my_sq()`) the normal way, and it does work on either a column or the whole DF
- Now why do we need `.apply()`?
    - we know functions can take arguments, maybe it does not work with arguments?
    - Look at the example below

In [6]:
# function def. - calculate square
def my_exp(x, e):
    return x ** e

In [7]:
my_exp(2, 3)

8

In [8]:
# it appears that taking parameters is not a problem
# let's come back to the 'why' part later
my_exp(df1['a'], 2)

0    100
1    400
2    900
Name: a, dtype: int64

# How `.apply()` works?

- `.apply()` is essentially a Series method 
    - which means natively we can _apply_ a function to a Series (column)
    - what `.apply()` does is that for every element in the series, the function is applied to it
        - and the results are returned as a Series of the same length

In [9]:
sq = df1['a'].apply(my_sq)
sq

0    100
1    400
2    900
Name: a, dtype: int64

In [10]:
cb = df1['a'].apply(my_exp, e=2)
cb

0    100
1    400
2    900
Name: a, dtype: int64

In [11]:
cb1 = []
for v in df1['a'].values:
    #print(v)
    cb1.append(my_exp(v, 2))
pd.Series(cb1)

0    100
1    400
2    900
dtype: int64

# What happened above?

- as you saw in these examples, `.apply()` works like with a `for` loop embedded
    - the function (e.g., `my_exp()`) is broadcasted to all the values in the Series (`df1['a']`)
    - and the return value is automatically converted to a `pandas.Series`
- this is how we avoid using `for` loops in `pandas`
    - as we said before, `for` loops are expensive, try avoiding them whenever you can
    - this is the first benefit of using `.apply()`

In [12]:
# we can do the same to a DF
# note that one different between `.apply()` and the regular function call is
# in `.apply()` you have to say explicitly what is the name of the argument (`e`)
df1.apply(my_exp, e=2)

,a,b
0,100,400
1,400,900
2,900,1600


In [13]:
# but if you try to apply a function with unmatched number of inputs
# it will raise an error - see this example

# this function takes three inputs
def avg_3(x, y, z):
    return (x + y + z) / 3

In [14]:
# when you apply the function to `df1` - since `df1` only has two columns
# this will raise an error
df1.apply(avg_3)

TypeError: ("avg_3() missing 2 required positional arguments: 'y' and 'z'", 'occurred at index a')

In [15]:
# consider the logic above - maybe we want to take the average of each column?
# we can rewrit the function like below
def avg_3_apply(col):
    x = col[0]
    y = col[1]
    z = col[2]
    return (x + y + z) / 3

In [16]:
# now it works
df1.apply(avg_3_apply)

a    20.0
b    30.0
dtype: float64

# Your Turn Here

Explain why above code works.

# `.apply()` Works on Columns Natively

- Above example shows an important thing
    - do you want to apply the funtion to each column or each row
    - natively `.apply()` works on columns
    - but you can change that by adding an argument `axis=0` so it applies on _rows_

- note that in `pandas`, `axis=0` always refers to rows, and `axis=1` to columns

In [17]:
df1.apply(avg_3_apply, axis=1)

IndexError: ('index out of bounds', 'occurred at index 0')

In [18]:
df1.apply(avg_3_apply, axis=0)

a    20.0
b    30.0
dtype: float64

In [19]:
# another example
def avg_2_apply(row):
    x = row[0]
    y = row[1]
    return (x + y) / 2

In [20]:
df1.apply(avg_2_apply, axis=1)

0    15.0
1    25.0
2    35.0
dtype: float64

In [21]:
# another way of doing this - note that this is much more expensive than `.apply()`
for index, row in df1.iterrows(): # `.iterrows()` iterate through rows in a DF
    # print(index, row)
    # break
    # index is the index value of the row
    print(index, avg_2_apply(row))

0 15.0
1 25.0
2 35.0


# A More Complex Example of `.apply()`

- So far we have been playing with a very simple DF
- We actually use `.apply()` for more complicated use cases
    - e.g., testing the _missingness_ in a dataset

In [4]:
# load a dataset
# the `titanic` dataset is one of the most popular dataset in analytics
import seaborn as sns
import pandas as pd
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [23]:
# in lecture 9, we had a way of calulating missingness
# count of missing values by column
titanic.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [5]:
# we can also calculate the ratio of missing values
(titanic.isna().sum()/titanic.shape[0]).round(4) * 100

survived        0.00
pclass          0.00
sex             0.00
age            19.87
sibsp           0.00
parch           0.00
fare            0.00
embarked        0.22
class           0.00
who             0.00
adult_male      0.00
deck           77.22
embark_town     0.22
alive           0.00
alone           0.00
dtype: float64

In [6]:
# we use np.sum() since you can only apply functions not methods
# `.sum()` as we used above is a method
import numpy as np
def count_missing(col):
    """Counts the number of missing values in a column
    """
    null_col = pd.isna(col)
    null_count = np.sum(null_col)
    return null_count

In [7]:
cmis_col = titanic.apply(count_missing)
cmis_col

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [9]:
cmis_row = titanic.apply(count_missing, axis=1)
cmis_row

0      1
1      0
2      1
3      0
4      1
      ..
886    1
887    0
888    2
889    0
890    1
Length: 891, dtype: int64

# Your Turn Here

Please explain the results of the above code block.

# Lambda Functions

- Regular Python functions require a definition, and a name
- Sometimes the function is so simple that it does not deserve a definition and a name
- we call them anonymous functions - which is __lambda__
    - _lambda_ has no name, and takes _arguments_ and specifies an _expression_ (usually an one-liner)
    - you do not have to specify the return value - it is automatic
- __lambda__ has a structure as following:

```python
lambda arguments: expression
```

In [110]:
# same as the my_exp function earlier
exp_lambda = lambda x, y: x ** y
exp_lambda(2, 3)

6

In [29]:
exp_lambda(3, 2)

9

In [30]:
exp_lambda(df1['a'], 2)

0    100
1    400
2    900
Name: a, dtype: int64

# When is the Best Time to use `lambda`?

- `lambda` is particularly useful when you deal with _lists_, _Series_, and _DataFrames_
    - In particular, when we need to transform a column in a DF
- the expression in `lambda` has to be simple enough
    - if the operation is complex, you can define it in a function, and use a lambda
    - if the operattion contains `if` statements or `for` loop, you should consider using a function rather than a `lambda`
- Being able to use `lambda` is the utmost benefit of using `.apply()`

In [31]:
# this is how you apply lambda to a column
df1['a'].apply(exp_lambda, y=2)

0    100
1    400
2    900
Name: a, dtype: int64

In [32]:
# an even easier way 
# you do not need any definition or function name
df1['a'].apply(lambda x: x**2)

0    100
1    400
2    900
Name: a, dtype: int64

In [10]:
# a complex function
def my_gender(x):
    if x == 'female':
        return 'f'
    else:
        return 'm'

In [11]:
# using lambda
titanic['sex'].apply(lambda x: my_gender(x)).head()

0    m
1    f
2    f
3    f
4    m
Name: sex, dtype: object

In [12]:
# equivalent of above
titanic['sex'].apply(my_gender).head()

0    m
1    f
2    f
3    f
4    m
Name: sex, dtype: object

# Other Ways to Use Functions in `pandas`

- `.map()` is another method
- difference between `.map()` and `.apply()` is that 
    - `.map()` can only work on a single Series (column)
    - `.apply()` can work on the whole DataFrame

In [36]:
df1.apply(lambda x: x**2)

,a,b
0,100,400
1,400,900
2,900,1600


In [37]:
# this will cause an error
df1.map(lambda x: x**2) # لازم يكون سيريز

AttributeError: 'DataFrame' object has no attribute 'map'

# Other Ways to Use Functions in `pandas`

- since `.map()` has limited usabilty, only one column, it is not very useful
- but we have a hybrid method `.applymap()`
    - which is the combination of `.map()` nad `.apply()`
    - reason of using `.applymap()` is that it is much faster comparing to `.apply()`, and also works on the whole DF

In [38]:
df1.applymap(lambda x: x**2)

,a,b
0,100,400
1,400,900
2,900,1600


# Popular Use Cases of `.apply()` and `lambda`

- We use the combination of `.apply()` and `lambda` in `pandas` when we are dealing with these scenarios
    - creating a new column based on an existing column
    - filtering a DataFrame (selecting a subset of columns)
    - extracting data from a column

In [13]:
# let's reading a dataset as an example
# please change your PATH to `'/srv/data/my_shared_data_folder/ba505-data/IMDB-Movie-Data.csv'`
imdb_data = pd.read_csv('/srv/data/my_shared_data_folder/ba505-data/IMDB-Movie-Data.csv')
imdb_data.head(2)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0


In [40]:
# we can calculate the average rating of a movie
# by average the `Rating` and a tenth of the `Metascore`
imdb_data['AvgRating'] = (imdb_data['Rating'] + imdb_data['Metascore']/10)/2
imdb_data['AvgRating'].head()

0    7.85
1    6.75
2    6.75
3    6.55
4    5.10
Name: AvgRating, dtype: float64

In [41]:
# we can filter the DF by the values of a certain column
# say we want to filter the `imdb_data` by the `Title` column
# if the column contains more than 4 words then we select them

long_title_movie_data = imdb_data[imdb_data['Title'].apply(lambda x: len(x.split())>=4)]
long_title_movie_data.head(3)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,AvgRating
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,7.85
8,9,The Lost City of Z,"Action,Adventure,Biography","A true-life drama, centering on British explor...",James Gray,"Charlie Hunnam, Robert Pattinson, Sienna Mille...",2016,141,7.1,7188,8.01,78.0,7.45
10,11,Fantastic Beasts and Where to Find Them,"Adventure,Family,Fantasy",The adventures of writer Newt Scamander in New...,David Yates,"Eddie Redmayne, Katherine Waterston, Alison Su...",2016,133,7.5,232072,234.02,66.0,7.05


In [14]:
name_df = pd.DataFrame(data = ['Braund, Mr. Owen Harris',
 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
 'Heikkinen, Miss. Laina',
 'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
 'Allen, Mr. William Henry',
 'Moran, Mr. James',
 'McCarthy, Mr. Timothy J',
 'Palsson, Master. Gosta Leonard',
 'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)',
 'Nasser, Mrs. Nicholas (Adele Achem)'], columns = ['Name'] )

#Take a look at the Data 
name_df.head(3)

,Name
0,"Braund, Mr. Owen Harris"
1,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
2,"Heikkinen, Miss. Laina"


In [43]:
# we observe the the title is always after the comma (`,`)
# and separate from the first name by a period (`.`)
# following code does the trick
 name_df['Name'].apply(lambda x: x.split(" ")[1].replace(".", ""))
name_df.head(3)

,Name,Title
0,"Braund, Mr. Owen Harris",Mr
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Mrs
2,"Heikkinen, Miss. Laina",Miss


# Your Turn Here
Finish exercises below by following instructions of each of them.

## Q1. Coding Problem

Complete excecises regarding data types of the given DataFrame (`itinery_df`).

In [1]:
import random
import pandas as pd
# generating the DF
duration_mins = pd.Series(random.sample(range(1, 1800), 20), name='duration_mins')
work_types = ['lecture', 'consulting', 'research']
work_type_series = pd.Series(random.choices(work_types, k=20), name='work_types')
locations = ['Beijing, China', 'London, England', 'Paris, France', 'Munich, Germany', 
             'Sydney, Australia', 'Mumbai, India', 'Madrid, Spain']
loc_series = pd.Series(random.choices(locations, k=20), name='locations')
hour_rates = pd.Series([round(random.uniform(10.0, 20.0), 2) for i in range(20)], name='hour_rates')
hour_rates.loc[random.sample(range(1, 20), 5)] = 'missing'
duration_mins.loc[random.sample(range(1, 20), 5)] = 'missing'
itinery_df = pd.concat([duration_mins, work_type_series, loc_series, hour_rates], axis=1)
#itinery_df['duration_mins'] = itinery_df['duration_mins'].astype(str)
itinery_df.head()

,duration_mins,work_types,locations,hour_rates
0,1259,research,"Beijing, China",16.25
1,630,consulting,"London, England",15.55
2,missing,consulting,"London, England",18.76
3,1666,consulting,"Paris, France",17.03
4,1107,research,"Sydney, Australia",12.18


## Part 1:

Use `.apply()` and `lambda` to create a new column `duration_hrs` by converting `duration_mins` to hours (divide by `60`).
- make sure you handle all `'missing'` values in `duration_mins` - use the average of the column to replace missing values.

In [2]:
duration_mins_mean = itinery_df[itinery_df['duration_mins']!= 'missing'].mean()[0]
duration_mins_mean

1045.3333333333333

In [3]:
itinery_df['duration_mins'].value_counts()

missing    5
29         1
1083       1
499        1
1367       1
630        1
1107       1
1616       1
814        1
1259       1
1353       1
861        1
1507       1
1666       1
737        1
1152       1
Name: duration_mins, dtype: int64

In [4]:

itinery_df['duration_mins'] = itinery_df['duration_mins'].apply(lambda x: duration_mins_mean if x  == 'missing' else x )
itinery_df['duration_hrs'] = itinery_df['duration_mins'].apply(lambda x: x/60 )
itinery_df


,duration_mins,work_types,locations,hour_rates,duration_hrs
0,1259.000000,research,"Beijing, China",16.25,20.983333
1,630.000000,consulting,"London, England",15.55,10.500000
2,1045.333333,consulting,"London, England",18.76,17.422222
3,1666.000000,consulting,"Paris, France",17.03,27.766667
4,1107.000000,research,"Sydney, Australia",12.18,18.450000
5,1045.333333,research,"Sydney, Australia",11.89,17.422222
6,1367.000000,research,"Sydney, Australia",missing,22.783333
7,1045.333333,consulting,"Madrid, Spain",15.9,17.422222
8,29.000000,lecture,"Munich, Germany",10.79,0.483333
9,737.000000,consulting,"Paris, France",17.84,12.283333


## Part 2:

Use `.apply()` and `lambda` to create two new columns `cities` and `countries`.

- `cities` refer to the first part in `locations` - before the `,`
- `countries` refer to the second part in `locations`
- note that there is a space after `,` that you need to remove

In [5]:
itinery_df['countries'] = itinery_df['locations'].apply(lambda x: x.split(", ")[1])
itinery_df.head()

,duration_mins,work_types,locations,hour_rates,duration_hrs,countries
0,1259.000000,research,"Beijing, China",16.25,20.983333,China
1,630.000000,consulting,"London, England",15.55,10.500000,England
2,1045.333333,consulting,"London, England",18.76,17.422222,England
3,1666.000000,consulting,"Paris, France",17.03,27.766667,France
4,1107.000000,research,"Sydney, Australia",12.18,18.450000,Australia


In [6]:
itinery_df['cities'] = itinery_df['locations'].apply(lambda x: x.split(", ")[0])
itinery_df.head()

,duration_mins,work_types,locations,hour_rates,duration_hrs,countries,cities
0,1259.000000,research,"Beijing, China",16.25,20.983333,China,Beijing
1,630.000000,consulting,"London, England",15.55,10.500000,England,London
2,1045.333333,consulting,"London, England",18.76,17.422222,England,London
3,1666.000000,consulting,"Paris, France",17.03,27.766667,France,Paris
4,1107.000000,research,"Sydney, Australia",12.18,18.450000,Australia,Sydney


## Part 3:

Use `.apply()` and `lambda` to create a column `work_load` using the following logic:

```python
if duration_hrs >= 20:
    # 'full_time'
else:
    # 'part_time'
```

In [7]:
def work_load_func(duration_hrs):
    if duration_hrs >= 20 :
        return'full_time'
    else:
        return'part_time'


In [8]:
itinery_df['work_load'] = itinery_df['duration_hrs'].apply(work_load_func)
itinery_df

,duration_mins,work_types,locations,hour_rates,duration_hrs,countries,cities,work_load
0,1259.000000,research,"Beijing, China",16.25,20.983333,China,Beijing,full_time
1,630.000000,consulting,"London, England",15.55,10.500000,England,London,part_time
2,1045.333333,consulting,"London, England",18.76,17.422222,England,London,part_time
3,1666.000000,consulting,"Paris, France",17.03,27.766667,France,Paris,full_time
4,1107.000000,research,"Sydney, Australia",12.18,18.450000,Australia,Sydney,part_time
5,1045.333333,research,"Sydney, Australia",11.89,17.422222,Australia,Sydney,part_time
6,1367.000000,research,"Sydney, Australia",missing,22.783333,Australia,Sydney,full_time
7,1045.333333,consulting,"Madrid, Spain",15.9,17.422222,Spain,Madrid,part_time
8,29.000000,lecture,"Munich, Germany",10.79,0.483333,Germany,Munich,part_time
9,737.000000,consulting,"Paris, France",17.84,12.283333,France,Paris,part_time


##  Part 4:

Use `.apply()` and `lambda` to calculate the total payment for each row, $ payment_{total} = duration_hr \times hour\_rate $.

In order to do that, you need to:
1. verify the `duration_hrs` and `hour_rates` are in the numerical (float) type.
2. handle all `'missing'` values in the `hour_rates` column - use the average of the column to replace missing values.
3. create a new column namely `payments`, then put the calculation results in it.

In [9]:
itinery_df.dtypes


duration_mins    float64
work_types        object
locations         object
hour_rates        object
duration_hrs     float64
countries         object
cities            object
work_load         object
dtype: object

In [10]:
hour_rates_mean = itinery_df[itinery_df['hour_rates']!= 'missing'].mean()[0]
hour_rates_mean

1001.7555555555556

In [11]:
itinery_df['hour_rates'] = itinery_df['hour_rates'].apply(lambda x: hour_rates_mean if x  == 'missing' else x )
itinery_df.dtypes

duration_mins    float64
work_types        object
locations         object
hour_rates       float64
duration_hrs     float64
countries         object
cities            object
work_load         object
dtype: object

In [12]:
pmt_lambda = lambda x, y: x * y

itinery_df['payments'] = pmt_lambda( itinery_df['duration_hrs'], itinery_df['hour_rates']  )
itinery_df

,duration_mins,work_types,locations,hour_rates,duration_hrs,countries,cities,work_load,payments
0,1259.000000,research,"Beijing, China",16.250000,20.983333,China,Beijing,full_time,340.979167
1,630.000000,consulting,"London, England",15.550000,10.500000,England,London,part_time,163.275000
2,1045.333333,consulting,"London, England",18.760000,17.422222,England,London,part_time,326.840889
3,1666.000000,consulting,"Paris, France",17.030000,27.766667,France,Paris,full_time,472.866333
4,1107.000000,research,"Sydney, Australia",12.180000,18.450000,Australia,Sydney,part_time,224.721000
5,1045.333333,research,"Sydney, Australia",11.890000,17.422222,Australia,Sydney,part_time,207.150222
6,1367.000000,research,"Sydney, Australia",1001.755556,22.783333,Australia,Sydney,full_time,22823.330741
7,1045.333333,consulting,"Madrid, Spain",15.900000,17.422222,Spain,Madrid,part_time,277.013333
8,29.000000,lecture,"Munich, Germany",10.790000,0.483333,Germany,Munich,part_time,5.215167
9,737.000000,consulting,"Paris, France",17.840000,12.283333,France,Paris,part_time,219.134667


## Part 5:

Create a new column `final_pay` using the following logic (note that `work_load`, `payments` and `final_pay` are column names):

```python

if work_load == 'full_time':
    final_pay = payment * 1.05
elif work_load == 'part_time':
    final_pay = payment * 0.95
```

In [13]:
def final_pay_func(itinery_df , work ,pmt):
    if itinery_df[work] == 'full_time' :
        return itinery_df[pmt] * 1.05
    else:
        return itinery_df[pmt] * 0.95
    

In [186]:
itinery_df['final_pay'] = itinery_df.apply(lambda x: final_pay_func(x, 'work_load', 'payments'), axis = 1)

itinery_df

,duration_mins,work_types,locations,hour_rates,duration_hrs,countries,cities,work_load,payments,final_pay
0,777.000000,lecture,"Mumbai, India",14.080000,12.950000,India,Mumbai,part_time,182.336000,173.219200
1,795.733333,research,"Beijing, China",13.290000,13.262222,China,Beijing,part_time,176.254933,167.442187
2,795.733333,lecture,"Munich, Germany",889.564444,13.262222,Germany,Munich,part_time,11797.601343,11207.721276
3,31.000000,research,"Madrid, Spain",16.390000,0.516667,Spain,Madrid,part_time,8.468167,8.044758
4,895.000000,lecture,"Mumbai, India",18.850000,14.916667,India,Mumbai,part_time,281.179167,267.120208
5,8.000000,lecture,"London, England",889.564444,0.133333,England,London,part_time,118.608593,112.678163
6,1516.000000,consulting,"Mumbai, India",14.660000,25.266667,India,Mumbai,full_time,370.409333,388.929800
7,1264.000000,lecture,"Paris, France",17.150000,21.066667,France,Paris,full_time,361.293333,379.358000
8,872.000000,research,"Sydney, Australia",13.140000,14.533333,Australia,Sydney,part_time,190.968000,181.419600
9,1585.000000,research,"Beijing, China",19.300000,26.416667,China,Beijing,full_time,509.841667,535.333750


# Classwork (start here in class)
You can start working on them right now:
- Read Chapter 9 in PD4E 
- If time permits, start in on your homework. 
- Ask questions when you need help. Use this time to get help from the professor!

# Homework (do at home)
The following is due before class next week:
  - Any remaining classwork from tonight
  - DataCamp “Speed efficient methods for iterating through a DataFrame” assignment

Note: All work on DataCamp is logged. Don't try to fake it!

Please email [me](mailto:jtao@fairfield.edu) if you have any problems or questions.

![DSB logo](img/Dolan.jpg)
# Apply Functions to Your DataFrame

## PD4E Chapter 9: Apply
### How do you read/manipulate/store data in Python?